Extract data from forms with Azure Document Intelligence \
https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/concept-custom?view=doc-intel-3.1.0&tabs=fott \
https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/FormRecognizer/rest/python-train-extract.md \
Training https://learn.microsoft.com/en-us/training/modules/work-form-recognizer/?source=recommendations

In [ ]:
print("Hello World!")

In [ ]:
# Speech to text
from openai import OpenAI
client = OpenAI()
audio_file_path = "..\\data\\speech\\yearwiththesaints_00_anonymous.wav" 
audio_file= open(audio_file_path, "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format="text"
)

print(transcript)

In [ ]:
# Summarize 
from openai import OpenAI
client = OpenAI()
user_prompt = f"Could you please summarize the transcript ### {transcript} ###"
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": user_prompt},
  ]
)

print(response.choices[0].message.content)

In [ ]:
response.choices[0].message.content

In [ ]:
# Async usage
import asyncio
import json
from openai import AsyncOpenAI

client = AsyncOpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key="My API Key",
)

async def show_json(obj):
    display(json.loads(obj.model_dump_json())) 

async def main() -> None:
    chat_completion = await client.chat.completions.create(
        messages=[{"role": "user", "content": "Say this is a test"}],
        model="gpt-3.5-turbo",
    )
    await show_json(chat_completion)


# asyncio.run(main())
await main()

In [ ]:
# Streaming Responses
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for part in stream:
    print(part.choices[0].delta.content or "")

In [ ]:
# The async client uses the exact same interface.
from openai import AsyncOpenAI

client = AsyncOpenAI()

stream = await client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
async for part in stream:
    print(part.choices[0].delta.content or "")